Installing Dependencies

In [1]:
!pip install kaggle


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Define column names since the CSV doesn’t have headers
column_names = ["target", "ids", "date", "flag", "user", "text"]

# Load dataset with latin-1 encoding
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "kazanova/sentiment140",
    "training.1600000.processed.noemoticon.csv",
    pandas_kwargs={
        "names": column_names,
        "encoding": "ISO-8859-1"
    }
)

print("First 5 records:\n", df.head())


/home/user/nci-masters/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_290/217021111.py:8: DeprecationWarning: Use dataset_load() instead of load_dataset(). load_dataset() will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:
    target         ids                          date      flag  \
0       0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY   
1       0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   
2       0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY   
3       0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   
4       0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY   

              user                                               text  
0  _TheSpecialOne_  @switchfoot http://twitpic.com/2y1zl - Awww, t...  
1    scotthamilton  is upset that he can't update his Facebook by ...  
2         mattycus  @Kenichan I dived many times for the ball. Man...  
3          ElleCTF    my whole body feels itchy and like its on fire   
4           Karoli  @nationwideclass no, it's not behaving at all....  


In [3]:
type(df)

pandas.core.frame.DataFrame

In [4]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Bad pipe message: %s [b'{"\xd4\xea\xeeR\x14\xefUU\x08&Z\xec\xf2\xe4\xcfB \xcf\xa2\xd3XG\xb2o\nt\t\x7f\x18 \x98"F8\xe3H\xeb\xa1\xe3mX mu\xda\x08C-', b'\x1a\xc0+\xc0/\xc0,\xc00\xcc\xa9\xcc\xa8\xc0\t\xc0\x13\xc0\n\xc0\x14\x13\x01\x13\x02\x13\x03\x01\x00\x05A\x00\x0b\x00\x02\x01\x00\xff\x01\x00\x01\x00\x00\x17\x00\x00\x00\x12\x00\x00\x00\x05\x00\x05\x01\x00\x00\x00\x00\x00\n\x00\x0c\x00\n\x11\xec\x00\x1d\x00\x17\x00\x18\x00\x19\x00\r\x00\x1a\x00\x18\x08\x04\x04\x03\x08\x07\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x05\x03\x06\x03\x02\x01\x02\x03\x00+\x00\x05\x04\x03\x04\x03\x03\x003\x04\xea\x04\xe8\x11\xec\x04\xc0\xe8\x18\x06\t\xf2{\xeda2\x1btC\xe7\xb4!RFA9C,L\xf7m\x04k\xb5hl\xb2c\xf4\xc2\x18i\x7f\x89\x96y\xb8a\xa6\x89,U\xb1wR\x19\x94\xad\x1c\x8a\xb0jQ^\xf0[\x9b\x19\xc7']
Bad pipe message: %s [b'k{l\xf8\x87N\xd1\xbd\xea\xecKQ\x18\x11L\xeci\xe2W\x1d\xcd\x84z\x08\x16\xc0\x92:P\x0f|n\xca[X\xae\xdbYqcD\x07\xf9\x9f\xbaJ\x8cB\xe3\x1f\xf9k\x7f\x819\xa3\x95z"\x1el\x0e\xeas?\xc5\xaa\x89\x13\xfa\xabbY

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stopwords.words('english')[:10]

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an']

In [7]:
a=pd.read_csv(r"/home/user/nci-masters/training.1600000.processed.noemoticon.csv",encoding="ISO-8859-1")
a.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [8]:
a=pd.read_csv(r"/home/user/nci-masters/training.1600000.processed.noemoticon.csv",names=column_names,encoding="ISO-8859-1")
a.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
a.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [10]:
a['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [11]:
a['target']=a['target'].replace(4,1)
a['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 -> Negative Tweet
1 --> Positive Tweet

Stemming

Stemming is the process of reducing a word to its Root word
example: actor, actress, acting = act


In [12]:
port_stem=PorterStemmer()
def stemmer(content):
    stemmed_content=re.sub('[^a-zA-z]',' ',content)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(i) for i in stemmed_content if not i in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

 

    

In [13]:
a['stemmed_content']=a['text'].apply(stemmer)

In [14]:
a.head()

,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [15]:
a['target']

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64

In [16]:
X=a['stemmed_content'].values

In [17]:
Y=a['target'].values

Splitting Data into Train and Test data 

In [18]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=42)

In [19]:
X_train

array(['paisleypaisley lol get idea far advanc even june yet need third knitter summer group',
       'worst headach ever',
       'ewaniesciuszko sad wont see miss alreadi yeah perfect come back th',
       ..., 'got home meet talk endlessli one coolest guy ever met smile',
       'bought chocol bar quot win free bar quot label win either',
       'misecia said hope dm email sunday'],
      shape=(1280000,), dtype=object)

In [20]:
X_test

array(['stm_denali ye black red fav color realli want color def look awesom jare',
       'qu buy open hous weekend pm best valu one bedroom lic long island citi bd http tinyurl com pt nqd',
       'ginoandfran fran greet air okay hahahaha thank', ...,
       'la__brat follow also hope atleast get also wish get well soon',
       'feel like decent swell sinc last fall hope wave myrtl beach week either least golf',
       'relaxin busi day'], shape=(320000,), dtype=object)

In [21]:
vectorizer=TfidfVectorizer()
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [22]:
model=LogisticRegression(max_iter=1000)

Training our model with training data

In [23]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Testing our model performance using accuracy metric

In [24]:
X_train_prediction=model.predict(X_train)
training_accuracy=accuracy_score(Y_train,X_train_prediction)
print(training_accuracy)

0.8062265625


In [25]:
X_test_prediction=model.predict(X_test)
testing_accuracy=accuracy_score(Y_test,X_test_prediction)
print(testing_accuracy)

0.77846875


In [26]:
X_train.shape

(1280000, 483642)

In [27]:
Y_train.shape

(1280000,)

In [28]:
X_test.shape

(320000, 483642)

In [29]:
Y_test.shape

(320000,)

Saving model in our system using pickel

In [30]:
import pickle
file_name="trained_model.sav"
pickle.dump(model,open(file_name,"wb"))

In [31]:
loaded_model=pickle.load(open('/home/user/nci-masters/twitter sentimental analysis/trained_model.sav','rb'))

In [32]:
X_new=X_test[300]
results=loaded_model.predict(X_new)
print(results)
if results[0]==0:
    print("Negitive Tweet")
else:
    print("Positive Tweet")

[1]
Positive Tweet
